In [3]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../../hand_made_stuff')
from handmadestuff import fit_predict_report

# Importing data
loan_df = pd.read_csv('data/loan.csv')

# Dropping rows containing NaN values
loan_df = loan_df.dropna().iloc[:, 1:]

In [17]:
loan_df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [18]:
X = loan_df.iloc[:, :11].values
y = loan_df.iloc[:, -1].values

In [19]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
for i in [0, 1, 2, 3, 4, 10]:
    X[:, i] = labelencoder_X.fit_transform(X[:, i])

In [20]:
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

In [21]:
X_1, X_2 = X[:, [0, 1, 2, 3, 4, 10]], X[:, [5, 6, 7, 8, 9]]

In [22]:
onehotencoder = OneHotEncoder()
X_1_encoded = onehotencoder.fit_transform(X_1[:, [2, 5]]).toarray()[:, 1:6]

In [23]:
X_1_not_encoded = X_1[:, [0, 1, 3, 4]]
X_1 = np.concatenate((X_1_not_encoded, X_1_encoded), axis = 1).astype('int')

In [24]:
X_1[0]

array([1, 1, 0, 0, 1, 0, 0, 1, 0])

In [25]:
X = np.concatenate((X_1, X_2.astype(int)), axis = 1)

In [26]:
# Splitting the dataset into the train and the test datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state = 0)

In [27]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

F:\STUDY\programming\anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [28]:
fit_predict_report(loan_df, 'knn', X_train, y_train, X_test, y_test)

Classificator used:
 knn


Confusion matrix: 

 [[ 6 29]
 [ 2 59]] 


Classification report:

              precision    recall  f1-score   support

          0       0.75      0.17      0.28        35
          1       0.67      0.97      0.79        61

avg / total       0.70      0.68      0.60        96



In [51]:
fit_predict_report(loan_df, 'svc', X_train, y_train, X_test, y_test)

Classificator used:
 svc


Confusion matrix: 

 [[12 23]
 [ 0 61]] 


Classification report:

              precision    recall  f1-score   support

          0       1.00      0.34      0.51        35
          1       0.73      1.00      0.84        61

avg / total       0.83      0.76      0.72        96



In [30]:
fit_predict_report(loan_df, 'lr', X_train, y_train, X_test, y_test)

Classificator used:
 lr


Confusion matrix: 

 [[12 23]
 [ 1 60]] 


Classification report:

              precision    recall  f1-score   support

          0       0.92      0.34      0.50        35
          1       0.72      0.98      0.83        61

avg / total       0.80      0.75      0.71        96

